Elasticity is defined by the following relation known as the Hooke's law:

$$
\sigma_{ij} = C_{ijkl} \cdot \varepsilon_{kl}
$$

which expanded becomes a 6 x 6 symmetric tensor 

$$
\begin{bmatrix}
\sigma_{11} \\
\sigma_{22} \\
\sigma_{33} \\
\sigma_{23} \\
\sigma_{31} \\
\sigma_{12} 
\end{bmatrix}
=
\begin{bmatrix}
C_{1111} & C_{1122} & C_{1133} & C_{1123} & C_{1131} & C_{1112} \\
C_{2211} & C_{2222} & C_{2233} & C_{2223} & C_{2231} & C_{2212} \\
C_{3311} & C_{3322} & C_{3333} & C_{3323} & C_{3331} & C_{3312} \\
C_{2311} & C_{2322} & C_{2333} & C_{2323} & C_{2331} & C_{2312} \\
C_{3111} & C_{3122} & C_{1333} & C_{3123} & C_{3131} & C_{3112} \\
C_{1211} & C_{1222} & C_{1233} & C_{1223} & C_{1231} & C_{1212} 
\end{bmatrix}

\begin{bmatrix}
\epsilon_{11} \\
\epsilon_{22} \\
\epsilon_{33} \\
2\epsilon_{23} \\
2\epsilon_{31} \\
2\epsilon_{12} 
\end{bmatrix}
$$

which in Voigt notation becomes

$$
\begin{bmatrix}
\sigma_{1} \\
\sigma_{2} \\
\sigma_{3} \\
\sigma_{4} \\
\sigma_{5} \\
\sigma_{6} 
\end{bmatrix}
=
\begin{bmatrix}
C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16} \\
C_{12} & C_{22} & C_{23} & C_{24} & C_{25} & C_{26} \\
C_{13} & C_{23} & C_{33} & C_{34} & C_{35} & C_{36} \\
C_{14} & C_{24} & C_{34} & C_{44} & C_{45} & C_{46} \\
C_{15} & C_{25} & C_{35} & C_{45} & C_{55} & C_{56} \\
C_{16} & C_{26} & C_{36} & C_{46} & C_{56} & C_{66} 
\end{bmatrix}

\begin{bmatrix}
\epsilon_{1} \\
\epsilon_{2} \\
\epsilon_{3} \\
2\epsilon_{4} \\
2\epsilon_{5} \\
2\epsilon_{6} 
\end{bmatrix}
$$

In [1]:
import json
import numpy as np
from dataclasses import dataclass, field, asdict


@dataclass
class ElasticTensor:
    """A class that represents an elastic tensor and calculate, store and
    print various derived properties."""

    Cij: np.ndarray  # stiffness tensor in GPa
    density: float  # density in g/cm3
    Sij: np.ndarray = field(init=False)  # compliance tensor
    K_voigt: float = field(init=False)
    K_reuss: float = field(init=False)
    K_hill: float = field(init=False)
    G_voigt: float = field(init=False)
    G_reuss: float = field(init=False)
    G_hill: float = field(init=False)
    universal_anisotropy: float = field(init=False)
    isotropic_poisson_ratio: float = field(init=False)
    isotropic_avg_vp: float = field(init=False)
    isotropic_avg_vs: float = field(init=False)

    def __post_init__(self):
        # Calculate the compliance tensor
        self.Sij = np.linalg.inv(self.Cij)

        # unpack the elastic constants to make the equations easier to read
        c11, c22, c33, c44, c55, c66 = np.diag(self.Cij)
        s11, s22, s33, s44, s55, s66 = np.diag(self.Sij)
        c12, c13, c23 = self.Cij[0, 1], self.Cij[0, 2], self.Cij[1, 2]
        s12, s13, s23 = self.Sij[0, 1], self.Sij[0, 2], self.Sij[1, 2]

        # Calculate the bulk modulus Voigt average
        self.K_voigt = 1/9 * ((c11 + c22 + c33) + 2*(c12 + c23 + c13))

        # Calculate the bulk modulus Reuss average
        self.K_reuss = 1 / ((s11 + s22 + s33) + 2*(s12 + s23 + s13))

        # Calculate bulk modulus VRH average
        self.K_hill = (self.K_voigt + self.K_reuss) / 2

        # Calculate the shear modulus Voigt average
        self.G_voigt = 1/15 * ((c11 + c22 + c33) - (c12 + c23 + c13)
                               + 3*(c44 + c55 + c66))

        # Calculate the shear modulus Reuss average
        self.G_reuss = 15 / (4*(s11 + s22 + s33) - 4*(s12 + s23 + s13)
                             + 3*(s44 + s55 + s66))

        # Calculate shear modulus VRH average
        self.G_hill = (self.G_voigt + self.G_reuss) / 2

        # Calculate the Universal elastic anisotropy
        self.universal_anisotropy = (5*(self.G_voigt / self.G_reuss)
                                     + (self.K_voigt / self.K_reuss) - 6)

        # Calculate the isotropic average Poisson ratio
        self.isotropic_poisson_ratio = ((3*self.K_hill - 2*self.G_hill)
                                        / (6*self.K_hill + 2*self.G_hill))

        # calculate the isotropic average Vp
        self.isotropic_avg_vp = np.around(np.sqrt((self.K_hill + 4/3 * self.G_hill) / self.density), decimals=4)

        # calculate the isotropic average Vs
        self.isotropic_avg_vs = np.around(np.sqrt(self.G_hill / self.density), decimals=4)

    def print_summary(self):
        print("ElasticTensor instance summary:")
        for k, v in asdict(self).items():
            print(f"{k}: {np.around(v, decimals=3)}")

    def __repr__(self) -> str:
        return ('This is an object of class ElasticTensor')
 
    def save_to_json(self, filename):
        with open(filename, 'w') as f:
            json.dump(asdict(self), f)



In [2]:
# density (g/cm3) at 1.5 GPa from Wang et al. (2015)
rho = 2.742

# Elastic stiffness tensor (in GPa) values as a matrix Mij at
# alpha quartz has six independent elastic constants: C11, C33, C12, C13, C14 and C44
Cij_aQtz = np.array(
    [[ 87.64,  6.99,  11.91, -17.19,   0. ,   0.  ],
     [  6.99, 87.64,  11.91,  17.19,   0. ,   0.  ],
     [ 11.91, 11.91, 107.20,   0. ,    0. ,   0.  ],
     [-17.19, 17.19,   0. ,   57.94,   0. ,   0.  ],
     [  0. ,   0. ,    0. ,    0. ,  57.94, -17.19],
     [  0. ,   0. ,    0. ,    0. , -17.19,  39.88]])

# cell parameters
cell_alpha_qtz = {
    'symmetry': '-3m',
    'unit_cell': [4.914, 4.913, 5.405],
    'angles': [90.0, 90.0, 120.0],
    'ref': 'TODO'}

In [3]:
alpha_quartz = ElasticTensor(Cij_aQtz, density=rho)

In [4]:
alpha_quartz

This is an object of class ElasticTensor

In [5]:
alpha_quartz.print_summary()

ElasticTensor instance summary:
Cij: [[ 87.64   6.99  11.91 -17.19   0.     0.  ]
 [  6.99  87.64  11.91  17.19   0.     0.  ]
 [ 11.91  11.91 107.2    0.     0.     0.  ]
 [-17.19  17.19   0.    57.94   0.     0.  ]
 [  0.     0.     0.     0.    57.94 -17.19]
 [  0.     0.     0.     0.   -17.19  39.88]]
density: 2.742
Sij: [[ 0.013 -0.002 -0.001  0.004  0.     0.   ]
 [-0.002  0.013 -0.001 -0.004  0.     0.   ]
 [-0.001 -0.001  0.01   0.     0.     0.   ]
 [ 0.004 -0.004 -0.     0.02   0.     0.   ]
 [ 0.     0.     0.     0.     0.02   0.009]
 [ 0.     0.     0.     0.     0.009  0.029]]
K_voigt: 38.233
K_reuss: 37.724
K_hill: 37.979
G_voigt: 47.93
G_reuss: 41.683
G_hill: 44.806
universal_anisotropy: 0.763
isotropic_poisson_ratio: 0.077
isotropic_avg_vp: 5.97
isotropic_avg_vs: 4.042


In [6]:
alpha_quartz.G_voigt

47.93

$$
V_P = \sqrt{\frac{K + 4/3\mu}{\rho}}
$$

In [7]:
alpha_quartz.isotropic_avg_vp

5.9698

In [8]:
alpha_quartz.isotropic_avg_vp / alpha_quartz.isotropic_avg_vs

1.4767959627943796

Any copyright is dedicated to the Public Domain.
https://creativecommons.org/publicdomain/zero/1.0/